In [44]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [45]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

13


In [46]:
xls_col_info = get_excel_col_header(f"{test_data_path}\SalesData1.xlsx")
xls_col_info

{'SalesInfo': {'StoreID': {'ColumnID': 'A'},
  'SalesYear': {'ColumnID': 'B'},
  'SalesValue': {'ColumnID': 'C'},
  'StoreLocation': {'ColumnID': 'D'},
  'Owner': {'ColumnID': 'E'}},
 'RegionInfo': {'ID': {'ColumnID': 'A'}, 'Location': {'ColumnID': 'B'}},
 'StoreInfo': {'ID': {'ColumnID': 'A'},
  'RegionID': {'ColumnID': 'B'},
  'StoreOwner': {'ColumnID': 'C'}},
 'SaleGrowthReport1': {'StoreID': {'ColumnID': 'A'},
  'AverageSales': {'ColumnID': 'B'},
  'MinSales': {'ColumnID': 'C'},
  'MaxSales': {'ColumnID': 'D'}},
 'SaleGrowthReport2': {'StoreLocation-SalesYear': {'ColumnID': 'A'},
  'Year': {'ColumnID': 'B'},
  'Location': {'ColumnID': 'C'},
  'AverageSales': {'ColumnID': 'D'},
  'MinSales': {'ColumnID': 'E'},
  'MaxSales': {'ColumnID': 'F'}}}

In [ ]:
xls_str = extract_excel_structure(f"{test_data_path}\SalesData1.xlsx")
xls_str

[{'SalesInfo': {'StoreID': {'ColumnID': 'A', 'ColumnValue': 11},
   'SalesYear': {'ColumnID': 'B', 'ColumnValue': 2020},
   'SalesValue': {'ColumnID': 'C', 'ColumnValue': 15789},
   'StoreLocation': {'ColumnID': 'D',
    'ColumnValue': '=IFERROR(INDEX(RegionInfo!B:B, MATCH(INDEX(StoreInfo!B:B, MATCH(A2, StoreInfo!A:A, 0)), RegionInfo!A:A, 0)), "")'},
   'Owner': {'ColumnID': 'E',
    'ColumnValue': '=IFERROR(INDEX(StoreInfo!C:C, MATCH(A2, StoreInfo!A:A, 0)), "")'}}},
 {'RegionInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': 1},
   'Location': {'ColumnID': 'B', 'ColumnValue': 'Halifax'}}},
 {'StoreInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': 11},
   'RegionID': {'ColumnID': 'B', 'ColumnValue': 1},
   'StoreOwner': {'ColumnID': 'C', 'ColumnValue': 'Ethan Carter'}}},
 {'SaleGrowthReport1': {<openpyxl.worksheet.formula.ArrayFormula at 0x29d6b3b25d0>: {'ColumnID': 'A',
    'ColumnValue': 11},
   'AverageSales': {'ColumnID': 'B',
    'ColumnValue': '=AVERAGEIF(SalesInfo!A:A, A2, SalesInfo!C:

In [4]:
sprdsht_info = extract_excel_structure(f"{test_data_path}\SalesData1.xlsx")
json.dumps(sprdsht_info)

TypeError: keys must be str, int, float, bool or None, not ArrayFormula